In [8]:
from lxml import etree
from lxml.builder import E

mx_graph_tree = etree.parse('./Example Files/mxgraph.xml')

In [57]:
# Get all features for the feature model
def get_features(mx_graph):
    features = {}

    for feature in mx_graph.xpath('''//root | 
                                     //concrete | 
                                     //abstract'''):
        feature_dict = dict(feature.items())
        
        if feature_dict and 'clon' not in feature_dict['id']:
            features[feature_dict['id']] = {
                'type': feature_dict['type'],
                'label': feature_dict['label'],
            }
            
    return features

features = get_features(mx_graph_tree)
len(features)

25

In [60]:
# Get all relations for the feature model
def get_relations(mx_graph):
    relations = []

    for relation in mx_graph_tree.xpath('''//rel_concrete_root | 
                                           //rel_abstract_root | 
                                           //rel_concrete_abstract | 
                                           //rel_abstract_abstract |
                                           //rel_concrete_concrete'''):

        relation_dict = dict(relation.items())
        relation_source_dict = dict(relation.find('.//mxCell').items())

        relations.append({
            'type': relation_dict['relType'],
            'source': relation_source_dict['source'],
            'target': relation_source_dict['target'],
        })
        
    return relations

relations = get_relations(mx_graph_tree)
len(relations)

27

In [67]:
# Normalize naming to be a continous numeric set
def normilez_naming(features, relations):
    feature_keys = features.keys()
    
    feature_mapping = dict(list(zip(feature_keys, range(1, len(feature_keys) + 1))))
    
    # Transform the features
    new_features = {}
    
    for k, v in feature_mapping.items():
        new_features[v] = features[k]
        
    # Transform the relations
    new_relations = []
    
    for relation in relations:        
        new_relations.append({
            'type': relation['type'],
            'source': feature_mapping[relation['source']],
            'target': feature_mapping[relation['target']],
        })
        
    return new_features, new_relations

features, relations = normilez_naming(features, relations)
len(features), len(relations)

(25, 27)

In [69]:
def get_root(features):
    for key, feature in features.items():
        if feature['type'] == 'root':
            return key

get_root(features)

1

In [83]:
problem_line = ["p cnf {} {}".format(len(features), len(relations))]

cnf_converter = {
    'root': lambda x: f'{x} 0',
    'optional': lambda x, y: f'-{x} {y} 0',
    'requires': lambda x, y: f'-{x} {y} 0',
    'excludes': lambda x, y: f'-{x} -{y} 0',
    'mandatory': lambda x, y: f'-{x} {y} 0 \n {y} {x} 0',
}

root_line = [cnf_converter['root'](get_root(features))]

constrain_lines = [
    cnf_converter[relation['type']](relation['source'], relation['target']) for relation in relations
]

In [84]:
print('\n'.join(problem_line + root_line + constrain_lines))

p cnf 25 27
1 0
-2 1 0
-3 1 0
-4 1 0 
 1 4 0
-5 1 0 
 1 5 0
-6 1 0
-8 9 0
-21 7 0
-19 15 0
-22 1 0
-10 22 0 
 22 10 0
-12 22 0
-11 22 0
-13 23 0
-16 23 0 
 23 16 0
-17 23 0
-14 23 0 
 23 14 0
-15 23 0
-23 1 0 
 1 23 0
-18 24 0 
 24 18 0
-19 24 0
-20 24 0
-21 24 0
-24 1 0
-9 25 0
-7 25 0 
 25 7 0
-8 25 0
-25 1 0
